# Bias base line

In [120]:
library(sdazar)
library(readxl)
library(lubridate)
library(stringr)
library(ggplot2)
library(patchwork)
library(ggrepel)
library(ggridges)
library(cobalt)
library(texreg)
source('utils/functions.R')

## Read administrative data 

In [121]:
# registro personas encuestadas
renc = data.table(read_excel('data/LISTA MUESTRA REGISTRO.xlsx', sheet=1))
setnames(renc, names(renc), tolower(names(renc)))

In [122]:
setnames(renc, c('folio', 'unidad', 'delito', 'mes egreso', 'tipo egreso'), 
        c('id', 'unidad', 'rdelito', 'mes_egreso', 'tegreso'))

In [123]:
renc[, condicional:=ifelse(tegreso %like% 'CONDICIONAL', 1, 0)]
renc[, cpf := ifelse(unidad %like% 'CPF', 1, 0)]

In [124]:
# registro no encuestadas
nores = data.table(read_excel('data/Datos de mujeres - Estudio Sociologia UC.xlsx', sheet=2))
setnames(nores, names(nores), tolower(names(nores)))
nores[, id := .I]

In [125]:
vars = c('id', 'fecha ingreso', 'fecha egreso', 'delito', 'extensión de condena', 
         'patio', 'unidad', 'n de condenas anteriores', 'año de nacimiento', 'edad al ingreso', 
         'comuna', 'estado civil', 'hijos (cantidad)', 'tipo egreso', 'razon de ausencia')

In [126]:
nores = nores[, vars, with=FALSE]

In [127]:
nvars = c('id', 'ingreso', 'fegreso', 'delito', 'condena', 
         'patio', 'unidad', 'ncondenas', 'nacimiento', 'edad_ingreso', 
         'comuna', 'estado_civil', 'nhijos', 'tegreso', 'razon')

setnames(nores, vars, nvars)

In [128]:
# new data for crime

pdel = fread('data/sesgo_delitos_poblacion.csv')
setnames(pdel, names(pdel), tolower(names(pdel)))

In [129]:
names(pdel)

[1] "id folio"             "nombre (s)"           "apellidos"           
 [4] "unidad penitenciaria" "fecha ingreso"        "delito"              
 [7] "fecha egreso"         "motivo egreso"        "mes de egreso"       
[10] "tipo población"

In [130]:
nvars = c('id', 'nombre', 'apellido', 'unidad', 'fingreso',  'rdelito', 'fegreso', 
          'razon_egreso', 'mes_egreso', 'muestra')

In [131]:
setnames(pdel, names(pdel), nvars)

In [132]:
table(pdel$muestra, useNA='ifany')


     MUESTRA NO POBLACIÓN    POBLACIÓN 
         100           16           16 

So the variables I would use are: 

- crime 
- unit
- number of sentences
- year born
- number of kids
- married
- kind of release

In [133]:
nores[, condicional:=ifelse(tegreso %like% 'CONDICIONAL', 1, 0)]
nores[, rechazo := ifelse(razon %like% 'RECHAZA', 1, 0)]

### Read Baseline

In [134]:
# read base line data
b = fread('data/170630 Base Reinserción Línea Base.csv')
setnames(b, names(b), tolower(names(b)))

In [135]:
#print(names(b))

In [136]:
vars = c('muestra', 'folio_2', 'p3_anio', 'p1', 'p5', 'p131', 'p13', 'p137')
nvars = c('muestra', 'id', 'nacimiento', 'edad', 'unidad', 'condena_anterior', 'nhijos', 'delito')
setnames(b, vars, nvars)

In [137]:
b = b[muestra==1]

In [138]:
length(b$id) == length(renc$id) # matched!

[1] TRUE

## Variable adjustments 

### married

In [139]:
b = assmis(b, list(c('p79', 'p60')), list(9))
b[, casada := pmax(p60, p79, na.rm=TRUE)][is.na(casada) & p49==0, casada:=0][is.na(casada), casada := 0]
prop.table(table(b$casada, useNA='ifany')) # lower proportion


   0    1 
0.92 0.08 

In [140]:
nores[, casada := ifelse(estado_civil=='Casada', 1, 0)]
prop.table(table(nores$casada, useNA='ifany'))


        0         1 
0.8846154 0.1153846 

### kids

In [141]:
table(b$nhijos, useNA='ifany')


 0  1  2  3  4  5  6  7  8  9 11 
24 51 47 45 27 10 13  5  1  1  1 

In [142]:
table(nores$nhijos, useNA='ifany')


 0  1  2  3  4  5  6  7  8 10 11 
 3 14  8  9  7  4  2  2  1  1  1 

### sentence before

In [143]:
nores[, condena_anterior := as.numeric(ncondenas>0)]
prop.table(table(nores$condena_anterior))


        0         1 
0.2884615 0.7115385 

In [144]:
b = assmis(b, list('condena_anterior'), list(9))
prop.table(table(b$condena_anterior))


        0         1 
0.2366071 0.7633929 

### year born

In [145]:
nores[, nac := as.numeric(nacimiento)]

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”

In [146]:
nores[is.na(nac), nac := year(ingreso)- as.numeric(edad_ingreso)]

In [147]:
summary(nores$nac, useNA = 'ifany')

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1938    1973    1982    1980    1991    1996 

In [148]:
b = assmis(b, list('nacimiento'), list(9999))
b[, nac := as.numeric(nacimiento)]
b[, ent_fecha := as.Date(fecha, "%m/%d/%Y")]
b[is.na(nac), nac := year(ent_fecha) - as.numeric(edad)]

In [149]:
summary(b$nac)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1948    1974    1982    1980    1989    1997 

### unit

In [150]:
table(nores$unidad)


       CPF TALITA KUM 
        49          3 

In [151]:
nores[, cpf := ifelse(unidad=='CPF', 1, 0)]

In [152]:
summary(nores$cpf)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  1.0000  1.0000  0.9423  1.0000  1.0000 

In [153]:
#b = assmis(b, list('unidad'), list(9))
#b[, cpf := ifelse(unidad==1, 1, 0)]
# summary(b$cpf)

In [154]:
setkey(renc, id)
setkey(b, id)

In [155]:
b[renc, cpf := i.cpf] # cool!

# type of release

In [156]:
table(nores$condicional, useNA='ifany')


 0  1 
41 11 

In [157]:
b[renc, condicional := condicional]

In [158]:
table(b$condicional)


  0   1 
156  69 

### crime

In [159]:
table(nores$delito)


        Asociación ilícita                    Estafas 
                         1                          1 
                 Homicidio                      Hurto 
                         1                          3 
              Hurto simple      Infracción Ley 17.336 
                        20                          1 
Lesiones graves gravísimas                 Ley 20.000 
                         1                         15 
              Microtráfico                Receptación 
                         1                          1 
        Robo con violencia  Robo en lugar no habitado 
                         1                          1 
         Robo por sorpresa      Sustracciones menores 
                         1                          1 
                   Tráfico 
                         3 

In [160]:
labels = sort(unique(nores$delito))

##### groups
1. Hurto
2. Otros delitos contra la propiedad
3. Delitos de drogas
4. Delitos contra las personas
5. Otro

In [161]:
labels

[1] "Asociación ilícita"         "Estafas"                   
 [3] "Homicidio"                  "Hurto"                     
 [5] "Hurto simple"               "Infracción Ley 17.336"     
 [7] "Lesiones graves gravísimas" "Ley 20.000"                
 [9] "Microtráfico"               "Receptación"               
[11] "Robo con violencia"         "Robo en lugar no habitado" 
[13] "Robo por sorpresa"          "Sustracciones menores"     
[15] "Tráfico"

In [162]:
codes = c(5,2,4,1,1,2,4,3,3,2,2,2,2,4,3)

In [163]:
nores[.(delito=labels, to=codes), on='delito', rdelito_g := i.to]

In [164]:
# classification
table(nores$delito,nores$rdelito_g, useNA = 'ifany')

                            
                              1  2  3  4  5
  Asociación ilícita          0  0  0  0  1
  Estafas                     0  1  0  0  0
  Homicidio                   0  0  0  1  0
  Hurto                       3  0  0  0  0
  Hurto simple               20  0  0  0  0
  Infracción Ley 17.336       0  1  0  0  0
  Lesiones graves gravísimas  0  0  0  1  0
  Ley 20.000                  0  0 15  0  0
  Microtráfico                0  0  1  0  0
  Receptación                 0  1  0  0  0
  Robo con violencia          0  1  0  0  0
  Robo en lugar no habitado   0  1  0  0  0
  Robo por sorpresa           0  1  0  0  0
  Sustracciones menores       0  0  0  1  0
  Tráfico                     0  0  3  0  0

In [165]:
# registro de encuestadas
labels = sort(unique(renc$rdelito))

In [166]:
schema = data.table(delitos = labels)

In [167]:
schema = schema[delitos!='P1']

In [168]:
schema[delitos %like% 'HURTO|Hurto', codes := 1]
schema[delitos %like% 'HOMICIDIO|Homicidio|LESIONES LEVES|MALTRATO|AMENAZAS', codes := 4]
schema[delitos %like% 'ESTAFA|Estafa|ROBO|PROPIEDAD INTELECTUAL|RECEPTACION', codes := 2]
schema[delitos %like% 'MICROTRAFICO|TRAFICO', codes := 3]
schema[delitos %like% 'Ley Emilia|PORTE ILEGAL', codes := 5]

In [169]:
renc[.(rdelito=schema$delitos, to=schema$codes), on='rdelito', rdelito_g := i.to]

In [170]:
# datos encuesta
table(b$delito, useNA = 'ifany')


 1  2  3  5  7  8  9 10 11 12 13 15 16 17 18 19 22 24 99 
 3  4  5  1 85  1  9 11 11  1  4 37 44  1  3  1  1  2  1 

In [171]:
lookvar(b, 'p137')

[1] "p137_otro"

In [172]:
b[p137_otro %in% c('LEY EMILIA'), delito :=13]
b[p137_otro %in% c('PARRICIDIO'), delito :=24]

In [173]:
codes = c(2,2,2,2,1,2,2,2,2,4,4,3,3,2,2,4,2,5,99)
dcodes = sort(unique(b$delito))

In [174]:
length(codes) == length(dcodes)

[1] TRUE

In [175]:
b[.(delito=dcodes, to=codes), on='delito', rdelito:=i.to]

In [176]:
table(b$delito, b$rdelito)

    
      1  2  3  4  5 99
  1   0  3  0  0  0  0
  2   0  4  0  0  0  0
  3   0  5  0  0  0  0
  5   0  1  0  0  0  0
  7  85  0  0  0  0  0
  8   0  1  0  0  0  0
  9   0  9  0  0  0  0
  10  0 11  0  0  0  0
  11  0 11  0  0  0  0
  12  0  0  0  1  0  0
  13  0  0  0  5  0  0
  15  0  0 37  0  0  0
  16  0  0 44  0  0  0
  17  0  1  0  0  0  0
  18  0  3  0  0  0  0
  19  0  0  0  1  0  0
  22  0  1  0  0  0  0
  24  0  0  0  0  1  0
  99  0  0  0  0  0  1

In [177]:
setkey(b, id)
b[renc, rdelito_g := rdelito_g]

Only 8 ids are shared in the data base, so I will keep using the old data!

In [178]:
test = unique(b[is.na(rdelito_g) & rdelito !=99]$id)

In [179]:
table(pdel$id %in% test)


FALSE  TRUE 
  124     8 

In [181]:
# impute missing crime data
b[is.na(rdelito_g) & rdelito !=99, rdelito_g := rdelito]

In [182]:
table(b$rdelito_g, useNA = 'ifany')


   1    2    3    4    5 <NA> 
  80   52   80    9    3    1 

In [183]:
# b = assmis(b, list('rdelito'), list(99))
prop.table(table(b$rdelito_g))


         1          2          3          4          5 
0.35714286 0.23214286 0.35714286 0.04017857 0.01339286 

In [184]:
prop.table(table(nores$rdelito))


         1          2          3          4          5 
0.44230769 0.11538462 0.36538462 0.05769231 0.01923077 

# Model rejection

In [185]:
m1 = glm(rechazo ~  nac + casada + nhijos + cpf + condena_anterior, 
         family='binomial', data = nores)

In [186]:
m2 = glm(rechazo ~  nac + as.factor(rdelito_g), 
         family='binomial', data = nores)

In [187]:
print(screenreg(list(m1, m2)))


                       Model 1   Model 2  
------------------------------------------
(Intercept)             15.15      -51.76 
                       (58.16)     (49.50)
nac                     -0.01        0.03 
                        (0.03)      (0.02)
casada                   0.75             
                        (1.07)            
nhijos                  -0.14             
                        (0.17)            
cpf                      1.39             
                        (1.40)            
condena_anterior        -1.58 *           
                        (0.70)            
as.factor(rdelito_g)2                0.71 
                                    (0.94)
as.factor(rdelito_g)3                1.08 
                                    (0.73)
as.factor(rdelito_g)4                0.21 
                                    (1.34)
as.factor(rdelito_g)5              -14.82 
                                 (1455.40)
------------------------------------------
AIC       

# Differences between groups

In [188]:
# merge dataset

vars = c('id', 'cpf', 'nac', 'rdelito_g', 'condicional', 
         'casada', 'nhijos', 'condena_anterior')
b1 = b[, vars, with=FALSE][, sample := 1]
n1 = nores[, vars, with=FALSE][, sample := 0]

dat = rbind(b1, n1)

In [189]:
nrow(dat)

[1] 277

In [190]:
table(dat$sample)


  0   1 
 52 225 

In [191]:
getdummies(dat, 'rdelito_g')

[[1]]
[1] "rdelito_g_1" "rdelito_g_3" "rdelito_g_2" "rdelito_g_4" "rdelito_g_5"



In [192]:
vars = paste0('rdelito_g_', 1:5)
nvars = c('del_hurto', 'del_propiedad', 'del_drogas', 'del_personas', 'del_otro')
setnames(dat, vars, nvars)

In [193]:
covars = c('cpf', 'nac', 'nhijos', 'condicional', 'condena_anterior', 'casada', nvars)
df = as.data.frame(dat[, covars, with=FALSE])
d = as.data.frame(rbm(df, as.factor(dat$sample)))

## Balance

In [194]:
d$labels = c('CPF', 'Año nacimiento', 'Num hijos', 'Libertad condicional', 
             'Condena anterior', 'Casada', 'Hurto', 'Delito propiedad', 'Delito drogas', 
            'Delito personas', 'Delito otro')

d$ind = as.numeric(abs(d$B) > .1)

In [195]:
plt = ggplot(d, aes(x=B, y=R, color=factor(ind))) + geom_point() +
    xlim(range(c(d$B[is.finite(d$B)], -1, 1))) +
    ylim(range(c(d$R[is.finite(d$R)], 0.1, 2))) + 
 geom_vline(xintercept = c(-0.1, 0.1), size=0.5, color='gray', linetype=3) + 
 geom_hline(yintercept = c(4/5, 5/4), size=0.5, color='gray', linetype=3) + 
 geom_text_repel(aes(B, R, label = labels), size=3) + 
 scale_color_manual(values=c('black', '#f03b20')) +
 labs(x='\nDiferencia estandarizada de promedios', y='Razón de varianza\n') +  
 theme_classic() + theme(legend.position='none')

In [196]:
savepdf('plots/balance')
print(plt)
dev.off()

pdf 
  2

In [197]:
options(repr.plot.width = 4, repr.plot.height = 3)

In [198]:
bal.tab(df[, c('condicional', 'nac', 'nhijos', 'condena_anterior', 'casada', 
              'del_propiedad', 'del_hurto')], treat=dat$sample, un=TRUE, 
        M.threshold='0.1', disp.means = TRUE, binary='raw', 
        s.d.denom='pooled', 
        disp.v.ratio=TRUE
       )

Balance Measures
                         Type    M.0.Un    M.1.Un Diff.Un V.Ratio.Un
condicional            Binary    0.2115    0.3067  0.0951           
nac                   Contin. 1979.5000 1979.7956  0.0239     1.4909
nhijos                Contin.    3.0769    2.5733 -0.2304     1.6258
condena_anterior       Binary    0.7115    0.7634  0.0519           
condena_anterior:<NA>  Binary    0.0000    0.0044  0.0044           
casada                 Binary    0.1154    0.0800 -0.0354           
del_propiedad          Binary    0.1154    0.2321  0.1168           
del_propiedad:<NA>     Binary    0.0000    0.0044  0.0044           
del_hurto              Binary    0.4423    0.3571 -0.0852           

Sample sizes
    Control Treated
All      52     225

# Standardized and relative bias

In [199]:
table(dat$sample)


  0   1 
 52 225 

In [200]:
# observed

ind = dat$sample

In [201]:
rs = dat[sample==1, covars, with=FALSE][, apply(.SD, 2, Mean)]
rp = dat[, covars, with=FALSE][, apply(.SD, 2, Mean)]

In [202]:
obs_r = (rs-rp)/rp

In [203]:
print(obs_r)

             cpf              nac           nhijos      condicional 
    4.327485e-03     2.802557e-05    -3.543527e-02     6.183333e-02 
condena_anterior           casada        del_hurto    del_propiedad 
    1.296360e-02    -7.666667e-02    -4.299584e-02     1.046798e-01 
      del_drogas     del_personas         del_otro 
   -4.329004e-03    -7.589286e-02    -7.589286e-02 


In [204]:
srd = dat[, covars, with=FALSE][, lapply(.SD, scale)]
ind = dat$sample == 1
srs = srd[ind,apply(.SD, 2, Mean)]
srp = srd[,apply(.SD, 2, Mean)]

In [205]:
obs_s = (srs - srp)

In [206]:
print(obs_s)

             cpf              nac           nhijos      condicional 
     0.021241968      0.004793208     -0.046774321      0.039332287 
condena_anterior           casada        del_hurto    del_propiedad 
     0.022631566     -0.023570381     -0.033115689      0.053896486 
      del_drogas     del_personas         del_otro 
    -0.003231698     -0.016151073     -0.009186673 


In [207]:
nrow(dat)

[1] 277

In [208]:
relbias = NULL
stdbias = NULL
for (i in 1:1000) { 
    ind = sample(nrow(dat), 225)
    
    rs = dat[ind, covars, with=FALSE][, apply(.SD, 2, Mean)]
    rp = dat[, covars, with=FALSE][, apply(.SD, 2, Mean)]
    
    srd = dat[, covars, with=FALSE][, lapply(.SD, scale)]
    srs = srd[ind,apply(.SD, 2, Mean)]
    srp = srd[,apply(.SD, 2, Mean)]
    
    s = (srs - srp)
    r = (rs-rp)/rp
    
    stdbias = rbind(stdbias, s)
    relbias = rbind(relbias, r)
    }

stdbias = as.data.table(stdbias)
relbias = as.data.table(relbias)

## Relative bias

In [209]:
covars = names(obs_r)

In [210]:
print(obs_r)

             cpf              nac           nhijos      condicional 
    4.327485e-03     2.802557e-05    -3.543527e-02     6.183333e-02 
condena_anterior           casada        del_hurto    del_propiedad 
    1.296360e-02    -7.666667e-02    -4.299584e-02     1.046798e-01 
      del_drogas     del_personas         del_otro 
   -4.329004e-03    -7.589286e-02    -7.589286e-02 


In [211]:
labels = c('CPF', 'Año nacimiento', 'Num hijos', 'Libertad condicional', 
             'Condena anterior', 'Casada', 'Hurto', 'Delito propiedad', 'Delito drogas', 
            'Delito personas', 'Delito otro')

In [212]:
for (i in seq_along(covars)) {     
    assign(paste0('plot_', i), ggplot(relbias, aes_string(x=covars[i])) +
          geom_density(color='white', alpha='0.3', fill='blue') +
          labs(y='', x= labels[i]) +
          geom_vline(xintercept = as.numeric(obs_r[covars[i]]), color='red', size=0.5, linetype=2) + 
          theme_classic())
}

In [213]:
options(repr.plot.width = 6, repr.plot.height = 8)
savepdf('plots/relative_bias', 16, 20)
eval(parse(text=paste0(paste0('plot_', seq_along(covars), collapse='+'), 
                       '+ plot_layout(ncol = 2)')))
dev.off()

pdf 
  2

# standardized bias

In [214]:
for (i in seq_along(covars)) {     
    assign(paste0('plot_', i), ggplot(stdbias, aes_string(x=covars[i])) +
          geom_density(color='white', alpha='0.3', fill='blue') +
         labs(y='', x= labels[i]) +
          geom_vline(xintercept = as.numeric(obs_s[covars[i]]), color='red', size=0.5, linetype=2) + 
          theme_classic())
}

In [215]:
options(repr.plot.width = 6, repr.plot.height = 8)
savepdf('plots/standardized_bias', 16, 20)
eval(parse(text=paste0(paste0('plot_', seq_along(covars), collapse='+'), 
                       '+ plot_layout(ncol = 2)')))
dev.off()

pdf 
  2

# Computing weights

In [218]:
library(ranger)

Warning message:
“package ‘ranger’ was built under R version 3.5.2”

In [219]:
print(names(dat))

 [1] "id"               "cpf"              "nac"              "rdelito_g"       
 [5] "condicional"      "casada"           "nhijos"           "condena_anterior"
 [9] "sample"           "del_hurto"        "del_drogas"       "del_propiedad"   
[13] "del_personas"     "del_otro"        


In [220]:
print(countmis(dat))

       rdelito_g condena_anterior        del_hurto       del_drogas 
           0.004            0.004            0.004            0.004 
   del_propiedad     del_personas         del_otro 
           0.004            0.004            0.004 


In [221]:
dat[is.na(cpf), cpf:=1]
dat[is.na(condena_anterior), condena_anterior:= 1]
dat[is.na(del_propiedad), del_propiedad:=0]
dat[is.na(del_drogas), del_drogas:=0]
dat[is.na(del_personas), del_personas:=0]
dat[is.na(del_otro), del_otro:=0]
dat[is.na(del_hurto), del_hurto:=0]

In [222]:
m = ranger(sample ~ nac + casada + nhijos + casada + condena_anterior + del_propiedad + + del_drogas + 
           del_personas + del_otro +  cpf,
           num.trees=1000,
           importance='permutation',
            data = dat)

In [223]:
print(sort(m$variable.importance, decreasing=TRUE))

             nac condena_anterior       del_drogas     del_personas 
    0.0043729586     0.0037069947     0.0031218087     0.0028854884 
          nhijos    del_propiedad         del_otro           casada 
    0.0020164038     0.0014072842    -0.0002485798    -0.0006852388 
             cpf 
   -0.0023824308 


In [224]:
probs = m$predictions

In [225]:
summary(probs)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4115  0.7525  0.8314  0.8089  0.8943  0.9689 

In [226]:
weights = 1 / probs

In [227]:
dat[, w := weights]

In [228]:
dat[sample==0, w := NA]

In [229]:
dat[, w := w / (sum(w, na.rm=TRUE)/sum(sample))]

In [230]:
sum(dat$w, na.rm=TRUE)

[1] 225

In [231]:
# using weights

In [232]:
rs = dat[sample==1, c(covars, 'w'), with=FALSE][, apply(.SD, 2, weighted.mean, w=w)]
rs = rs[-length(rs)]
rp = dat[, covars, with=FALSE][, apply(.SD, 2, Mean)]
obs_r = (rs-rp)/rp

In [233]:
for (i in seq_along(covars)) {     
    assign(paste0('plot_', i), ggplot(relbias, aes_string(x=covars[i])) +
          geom_density(color='white', alpha='0.3', fill='blue') +
          labs(y='', x= labels[i]) +
          geom_vline(xintercept = as.numeric(obs_r[covars[i]]), color='red', size=0.5, linetype=2) + 
          theme_classic())
}

In [234]:
options(repr.plot.width = 6, repr.plot.height = 8)
savepdf('plots/relative_bias_w', 16, 20)
eval(parse(text=paste0(paste0('plot_', seq_along(covars), collapse='+'), 
                       '+ plot_layout(ncol = 2)')))
dev.off()

pdf 
  2

In [235]:
srd = dat[, c(covars), with=FALSE][, lapply(.SD, scale)]
ind = dat$sample == 1

srd[, w := dat$w]

In [236]:
srs = srd[ind,apply(.SD, 2, weighted.mean, w=w)]
srs = srs[-length(srs)]

In [237]:
srp = srd[,apply(.SD, 2, Mean)]

srp = srp[-length(srp)]

In [238]:
obs_s = (srs - srp)

In [239]:
for (i in seq_along(covars)) {     
    assign(paste0('plot_', i), ggplot(stdbias, aes_string(x=covars[i])) +
          geom_density(color='white', alpha='0.3', fill='blue') +
         labs(y='', x= labels[i]) +
          geom_vline(xintercept = as.numeric(obs_s[covars[i]]), color='red', size=0.5, linetype=2) + 
          theme_classic())
}

In [240]:
options(repr.plot.width = 6, repr.plot.height = 8)
savepdf('plots/standardized_bias_w', 16, 20)
eval(parse(text=paste0(paste0('plot_', seq_along(covars), collapse='+'), 
                       '+ plot_layout(ncol = 2)')))
dev.off()

pdf 
  2